<a href="https://colab.research.google.com/github/qAp/soil/blob/develop/nbs/00_soilsample_covariates_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merging environmental covariates and soil samples

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap']) 


import time
import json
from pathlib import Path
import numpy as np
import pandas as pd
import ee
import geemap    

from google.colab import files

geemap package not installed. Installing ...


In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=I4Anm769AET4LM6rkkFLgVcYjEPTqPrb3hNYDL6pFzo&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5NtISw1jFdXSd-nCHZDvSX6SR7W00MNl0_H5gsg7CaxShEMxvEepY

Successfully saved authorization token.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data

## Soil samples

In [4]:
# soil_samples = ee.FeatureCollection("users/bingosaucer/orgc_stock_ISRIC_WISE_clip_gee")
soil_samples = ee.FeatureCollection("users/bingosaucer/orgc_stock_ISCN_clip_gee")

## Environmental covariates

In [5]:
! wget https://www.dropbox.com/s/et2nrdsx8bflpt4/environmental_covariate.csv

covariates = pd.read_csv('environmental_covariate.csv')
covariates.dropna(axis=0, subset=['Variable', 'GEE ID'], inplace=True)
print(covariates.shape)
covariates.head()

--2021-06-03 03:17:21--  https://www.dropbox.com/s/et2nrdsx8bflpt4/environmental_covariate.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/et2nrdsx8bflpt4/environmental_covariate.csv [following]
--2021-06-03 03:17:21--  https://www.dropbox.com/s/raw/et2nrdsx8bflpt4/environmental_covariate.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8c3cfcfedcd93b001549631955.dl.dropboxusercontent.com/cd/0/inline/BPuwXVigNP_bLtyM8GTWRmWaSPoUcE1gl7M6ik9PxfZzFdpaRBl9_VPTSdoPv6VyGWF11eTuFFu4OlpanBMUxQnS9GEhUjoyzqZDNj3hSrkkQSIc8X2rLjx-t5fx041igme5aDDjJ-UVRKWf3kYHOM0o/file# [following]
--2021-06-03 03:17:21--  https://uc8c3cfcfedcd93b001549631955.dl.dropboxusercontent.com/cd/0/inline/BPuwXVigNP_bLtyM8GTWRmWaSPoUcE1gl7M6ik9PxfZzF

,#,Variable,Description,Source / Link,doi,File Size (GB),GEE ID,Unnamed: 7,filename
4,5.0,EarthEnvCloudCover_MODCF_interannualSD,Mean between-year seasonality represented as t...,https://data.earthenv.org/cloud/MODCF_interann...,10.1371/journal. pbio.1002415,0.64,users/bingosaucer/EarthEnvCloudCover_MODCF_int...,NaN,NaN
5,6.0,EarthEnvCloudCover_MODCF_intraannualSD,Within-year seasonality represented as the sta...,https://data.earthenv.org/cloud/MODCF_intraann...,10.1371/journal. pbio.1002415,0.67,users/bingosaucer/EarthEnvCloudCover_MODCF_int...,NaN,NaN
6,7.0,EarthEnvCloudCover_MODCF_meanannual,Mean annual cloud frequency (%) over 2000-2014;,https://data.earthenv.org/cloud/MODCF_meanannu...,10.1371/journal. pbio.1002415,0.67,users/bingosaucer/EarthEnvCloudCover_MODCF_mea...,NaN,NaN
7,8.0,EarthEnvCloudCover_MODCF_seasonality_concentra...,Seasonal cloud concentration index (see method...,https://data.earthenv.org/cloud/MODCF_seasonal...,10.1371/journal. pbio.1002415,0.67,users/bingosaucer/EarthEnvCloudCover_MODCF_sea...,NaN,NaN
8,9.0,EarthEnvCloudCover_MODCF_seasonality_theta,Timing of peak seasonal cloud concentration (s...,https://data.earthenv.org/cloud/MODCF_seasonal...,10.1371/journal. pbio.1002415,0.67,users/bingosaucer/EarthEnvCloudCover_MODCF_sea...,NaN,NaN


# Merge workflow

In [6]:
Path.ls = lambda src: list(src.iterdir())

In [7]:
variables = ['EarthEnvCloudCover_MODCF_interannualSD', 'SG_Silt_Content_015cm']

In [8]:
geeids = covariates[covariates.Variable.isin(variables)]['GEE ID'].values
geeids

array(['users/bingosaucer/EarthEnvCloudCover_MODCF_interannualSD',
       'users/danvinci/ISRIC/SG_Silt_Content_015cm'], dtype=object)

In [9]:
imgs = [ee.Image(geeid) for geeid in geeids]
imgcollection = ee.ImageCollection(imgs)

In [10]:
def collection2image_stackbands(collection):
    '''
    Merge images in an image collection to a single image.  Each
    image in the collection becomes a band in the output image.
    
    Args:
        collection (ee.ImageCollection): Image collection.
    '''
    img = collection.iterate(lambda curr, prev: ee.Image(prev).addBands(curr), ee.Image(1))
    img = ee.Image(img)
    ibands = ee.List.sequence(1, img.bandNames().size().subtract(1))
    return img.select(ibands)

In [11]:
img = collection2image_stackbands(imgcollection)
img = img.rename(variables)
img.bandNames().getInfo()

['EarthEnvCloudCover_MODCF_interannualSD', 'SG_Silt_Content_015cm']

In [12]:
Map = geemap.Map(center=(40, -100), zoom=2)
for variable in variables:
    Map.addLayer(img.select(variable), 
                {'palette': ['#ffffcc', '#e31a1c', '#bd0026', '#800026'], 
                 'min':0, 'max':100},
                variable, True, 0.3)
Map.addLayer(soil_samples, {}, "soil_samples")
Map

In [13]:
result = img.reduceRegions(soil_samples, ee.Reducer.mean())
result.size().getInfo(), result.first().getInfo()

(32393,
 {'geometry': {'coordinates': [-103.14140190346887, 46.351347693731825],
   'type': 'Point'},
  'id': '00000000000000006b80',
  'properties': {'EarthEnvCloudCover_MODCF_interannualSD': 1100,
   'SG_Silt_Content_015cm': 45,
   'bot_cm': 30,
   'datetime': 1964,
   'mineral_fraction': 2,
   'organic_fraction': 0,
   'orgc_stock_mineral': 7.006440000000001,
   'orgc_stock_organic': 0,
   'profile_id': '64ND087002_40A1382',
   'top_cm': 0},
  'type': 'Feature'})

Exporting to Google Drive as CSV file.

In [18]:
csv_name = 'test_dataset'

In [19]:
pth = Path('/content/drive/MyDrive') / f'{csv_name}.csv'
if pth.exists():
    pth.unlink()

task = ee.batch.Export.table.toDrive(
    collection=result, description=csv_name, fileFormat='CSV')

task.start()

In [20]:
while task.active():
    print(f'Polling for task (id: {task.id}).')
    time.sleep(5)

Polling for task (id: 2CF2CMJRYC7HN7O6XYPKUDOR).
Polling for task (id: 2CF2CMJRYC7HN7O6XYPKUDOR).


Load saved CSV into dataframe and replace the `.geo` column with `longitude` and `latitude` columns.

In [21]:
df = pd.read_csv(pth)

json.loads(df['.geo'].iloc[0])['coordinates']
df['longitude'] = df['.geo'].apply(lambda x: json.loads(x)['coordinates'][0])
df['latitude'] = df['.geo'].apply(lambda x: json.loads(x)['coordinates'][1])

df.drop(['system:index', '.geo'], axis=1, inplace=True)

In [22]:
df.head()

,EarthEnvCloudCover_MODCF_interannualSD,SG_Silt_Content_015cm,bot_cm,datetime,mineral_fraction,organic_fraction,orgc_stock_mineral,orgc_stock_organic,profile_id,top_cm,longitude,latitude
0,1100.0,45.0,30.0,1964,2.0,0.0,7.00644,0.0,64ND087002_40A1382,0.0,-103.141402,46.351348
1,993.0,60.0,30.0,1966,2.0,0.0,14.23080,0.0,S1966IL107001_40A2008,0.0,-89.326942,40.263609
2,1058.0,73.0,30.0,1981,2.0,0.0,5.43600,0.0,1981IL133003_83P0357,0.0,-90.256392,38.240551
3,988.0,64.0,30.0,1987,2.0,0.0,17.41257,0.0,1987IL109068_88P0084,0.0,-90.689451,40.491661
4,945.0,62.0,30.0,1988,2.0,0.0,12.13996,0.0,1988IL067025_88P0797,0.0,-91.031389,40.547217


# Merging selected sets of covariates 

Check how this workflow works for selected sets of covariates, such as EarthEnv, 

## Consensus landcover

In [23]:
def consensus_landcover_labels():
    '''
    https://www.earthenv.org/landcover
    
    Returns: Global 1-km Consensus Land Cover classes
    '''
    return ['Evergreen/Deciduous Needleleaf Trees', 'Evergreen Broadleaf Trees', 'Deciduous Broadleaf Trees', 
            'Mixed/Other Trees', 'Shrubs', 'Herbaceous Vegetation', 'Cultivated and Managed Vegetation',
            'Regularly Flooded Vegetation', 'Urban/Built-up', 'Snow/Ice', 'Barren', 'Open Water']

def consensus_landcover_geeid():
    return 'users/bingosaucer/ConsensusLandCover_Human_Development_Percentage'

In [25]:
imgcollection = ee.ImageCollection(consensus_landcover_geeid())
img = collection2image_stackbands(imgcollection)
img = img.rename(consensus_landcover_labels())
result = img.reduceRegions(soil_samples, ee.Reducer.mean())
result.size().getInfo(), result.first().getInfo()

(32393,
 {'geometry': {'coordinates': [-103.14140190346887, 46.351347693731825],
   'type': 'Point'},
  'id': '00000000000000006b80',
  'properties': {'Barren': 0,
   'Cultivated and Managed Vegetation': 11,
   'Deciduous Broadleaf Trees': 0,
   'Evergreen Broadleaf Trees': 0,
   'Evergreen/Deciduous Needleleaf Trees': 0,
   'Herbaceous Vegetation': 0,
   'Mixed/Other Trees': 0,
   'Open Water': 0,
   'Regularly Flooded Vegetation': 0,
   'Shrubs': 0,
   'Snow/Ice': 0,
   'Urban/Built-up': 89,
   'bot_cm': 30,
   'datetime': 1964,
   'mineral_fraction': 2,
   'organic_fraction': 0,
   'orgc_stock_mineral': 7.006440000000001,
   'orgc_stock_organic': 0,
   'profile_id': '64ND087002_40A1382',
   'top_cm': 0},
  'type': 'Feature'})

## Topographical variables

In [26]:
is_topo = covariates.Variable.apply(lambda x: x.startswith('EarthEnvTopoMed_'))

df = covariates[is_topo].reset_index(drop=True)
print(df.shape)

(16, 9)


In [27]:
imgs = [ee.Image(geeid) for geeid in df['GEE ID'].values]
imgcollection = ee.ImageCollection(imgs)
img = collection2image_stackbands(imgcollection)
img = img.rename(df['Variable'].tolist())
result = img.reduceRegions(soil_samples, ee.Reducer.mean())
result.size().getInfo(), result.first().getInfo()

(32393,
 {'geometry': {'coordinates': [-103.14140190346887, 46.351347693731825],
   'type': 'Point'},
  'id': '00000000000000006b80',
  'properties': {'EarthEnvTopoMed_1stOrderPartialDerivEW': -0.00038976711221039295,
   'EarthEnvTopoMed_1stOrderPartialDerivNS': 0.00809656921774149,
   'EarthEnvTopoMed_2ndOrderPartialDerivEW': -2.462456905050203e-10,
   'EarthEnvTopoMed_2ndOrderPartialDerivNS': -3.263154940214008e-05,
   'EarthEnvTopoMed_AspectCosine': 0.8275126218795776,
   'EarthEnvTopoMed_AspectSine': -0.01998455449938774,
   'EarthEnvTopoMed_Eastness': -0.01396926585584879,
   'EarthEnvTopoMed_Elevation': 856.5,
   'EarthEnvTopoMed_Northness': 0.4282548129558563,
   'EarthEnvTopoMed_ProfileCurvature': -3.370612830622122e-05,
   'EarthEnvTopoMed_Roughness': 9,
   'EarthEnvTopoMed_Slope': 0.819472074508667,
   'EarthEnvTopoMed_TangentialCurvature': 1.9312163203721866e-05,
   'EarthEnvTopoMed_TerrainRuggednessIndex': 3.125,
   'EarthEnvTopoMed_TopoPositionIndex': -0.625,
   'EarthEnvT

## Soil properties (Soil Grid)

In [28]:
prefixes = ['SG_Clay', 'SG_Silt']
is_sg = covariates.Variable.apply(
    lambda x: any(x.startswith(prefix) for prefix in prefixes))
df = covariates[is_sg].reset_index(drop=True)
df.shape

(14, 9)

In [29]:
imgs = [ee.Image(geeid) for geeid in df['GEE ID'].values]
imgcollection = ee.ImageCollection(imgs)
img = collection2image_stackbands(imgcollection)
img = img.rename(df.Variable.tolist())
result = img.reduceRegions(soil_samples, ee.Reducer.mean())
result.size().getInfo(), result.first().getInfo()

(32393,
 {'geometry': {'coordinates': [-103.14140190346887, 46.351347693731825],
   'type': 'Point'},
  'id': '00000000000000006b80',
  'properties': {'SG_Clay_Content_000cm': 33,
   'SG_Clay_Content_005cm': 33,
   'SG_Clay_Content_015cm': 35,
   'SG_Clay_Content_030cm': 36,
   'SG_Clay_Content_060cm': 34,
   'SG_Clay_Content_100cm': 33,
   'SG_Clay_Content_200cm': 32,
   'SG_Silt_Content_000cm': 49,
   'SG_Silt_Content_005cm': 50,
   'SG_Silt_Content_015cm': 49,
   'SG_Silt_Content_030cm': 49,
   'SG_Silt_Content_060cm': 50,
   'SG_Silt_Content_100cm': 50,
   'SG_Silt_Content_200cm': 50,
   'bot_cm': 30,
   'datetime': 1964,
   'mineral_fraction': 2,
   'organic_fraction': 0,
   'orgc_stock_mineral': 7.006440000000001,
   'orgc_stock_organic': 0,
   'profile_id': '64ND087002_40A1382',
   'top_cm': 0},
  'type': 'Feature'})

In [ ]:
# ISRIC sample with NaN covariate values

# point = ee.Geometry.Point(-79.5, 8.95)
# point_buffer = point.buffer(distance=2000)

# Map = geemap.Map(center=(8.95, -79.5), zoom=6)
# for var in df.Variable.tolist():
#     Map.addLayer(img.select(var), {'palette': ['#ffffcc', '#e31a1c', '#bd0026', '#800026']}, 
#                  var, True, 0.3)

# Map.addLayer(wise3, {}, "WISE3")

# Map.addLayer(point, {'color':'black'}, 'NaN point')
# Map.addLayer(point_buffer, {'color':'red'}, 'NaN point buffer')
# Map

# Merging all available covariates in one go

In [30]:
# `MODIS_*` assets are large image collections, which will be left out for now, 
# because their images might need to be reduced first before merging. 

is_MODIS = covariates['GEE ID'].apply(lambda x: x.startswith('MODIS/'))
df = covariates.drop(covariates[is_MODIS].index, axis=0)

In [31]:

imgs, valid_ids = [], []
for geeid in df['GEE ID'].values:
    try:
        imgs.append(ee.Image(geeid))
        valid_ids.append(geeid)
    except:
        continue


imgcollection = ee.ImageCollection(imgs)
img = collection2image_stackbands(imgcollection)

img = img.rename(df.Variable.tolist())
result = img.reduceRegions(soil_samples, ee.Reducer.mean())

In [ ]:
%%time
result.size().getInfo(), result.first().getInfo()

In [33]:
csv_name = 'iscn_env_dataset'

In [34]:
pth = Path('/content/drive/MyDrive') / f'{csv_name}.csv'

if pth.exists():
    pth.unlink()

task = ee.batch.Export.table.toDrive(
    collection=result, description=csv_name, fileFormat='CSV')

task.start()

In [35]:
while task.active():
    print(f'Polling for task (id: {task.id}).')
    time.sleep(5)

Polling for task (id: K4DASCVGIXHPSUTUF7XA47HJ).
Polling for task (id: K4DASCVGIXHPSUTUF7XA47HJ).
Polling for task (id: K4DASCVGIXHPSUTUF7XA47HJ).
Polling for task (id: K4DASCVGIXHPSUTUF7XA47HJ).
Polling for task (id: K4DASCVGIXHPSUTUF7XA47HJ).
Polling for task (id: K4DASCVGIXHPSUTUF7XA47HJ).
Polling for task (id: K4DASCVGIXHPSUTUF7XA47HJ).
Polling for task (id: K4DASCVGIXHPSUTUF7XA47HJ).


In [36]:
df = pd.read_csv(pth)

json.loads(df['.geo'].iloc[0])['coordinates']
df['longitude'] = df['.geo'].apply(lambda x: json.loads(x)['coordinates'][0])
df['latitude'] = df['.geo'].apply(lambda x: json.loads(x)['coordinates'][1])

df.drop(['system:index', '.geo'], axis=1, inplace=True)

In [37]:
df.head()

,EarthEnvCloudCover_MODCF_interannualSD,EarthEnvCloudCover_MODCF_intraannualSD,EarthEnvCloudCover_MODCF_meanannual,EarthEnvCloudCover_MODCF_seasonality_concentration,EarthEnvCloudCover_MODCF_seasonality_theta,EarthEnvTopoMed_1stOrderPartialDerivEW,EarthEnvTopoMed_1stOrderPartialDerivNS,EarthEnvTopoMed_2ndOrderPartialDerivEW,EarthEnvTopoMed_2ndOrderPartialDerivNS,EarthEnvTopoMed_AspectCosine,EarthEnvTopoMed_AspectSine,EarthEnvTopoMed_Eastness,EarthEnvTopoMed_Elevation,EarthEnvTopoMed_Northness,EarthEnvTopoMed_ProfileCurvature,EarthEnvTopoMed_Roughness,EarthEnvTopoMed_Slope,EarthEnvTopoMed_TangentialCurvature,EarthEnvTopoMed_TerrainRuggednessIndex,EarthEnvTopoMed_TopoPositionIndex,EarthEnvTopoMed_VectorRuggednessMeasure,FanEtAl_Depth_to_Water_Table_AnnualMean,SG_Absolute_depth_to_bedrock,SG_Bulk_density_000cm,SG_Bulk_density_005cm,SG_Bulk_density_015cm,SG_Bulk_density_030cm,SG_Bulk_density_060cm,SG_Bulk_density_100cm,SG_Bulk_density_200cm,SG_CEC_000cm,SG_CEC_005cm,SG_CEC_015cm,SG_CEC_030cm,SG_CEC_060cm,SG_CEC_100cm,SG_CEC_200cm,SG_Clay_Content_000cm,SG_Clay_Content_005cm,SG_Clay_Content_015cm,...,WorldClim2_BIO_Mean_Diurnal_Range,WorldClim2_BIO_Mean_Temperature_of_Coldest_Quarter,WorldClim2_BIO_Mean_Temperature_of_Driest_Quarter,WorldClim2_BIO_Mean_Temperature_of_Warmest_Quarter,WorldClim2_BIO_Mean_Temperature_of_Wettest_Quarter,WorldClim2_BIO_Min_Temperature_of_Coldest_Month,WorldClim2_BIO_Precipitation_Seasonality,WorldClim2_BIO_Precipitation_of_Coldest_Quarter,WorldClim2_BIO_Precipitation_of_Driest_Month,WorldClim2_BIO_Precipitation_of_Driest_Quarter,WorldClim2_BIO_Precipitation_of_Warmest_Quarter,WorldClim2_BIO_Precipitation_of_Wettest_Month,WorldClim2_BIO_Precipitation_of_Wettest_Quarter,WorldClim2_BIO_Temperature_Annual_Range,WorldClim2_BIO_Temperature_Seasonality,WorldClim2_H2OVaporPressure_AnnualMean,WorldClim2_SolarRadiation_AnnualMean,WorldClim2_WindSpeed_AnnualMean,bot_cm,datetime,humandev_consensus_full_class_1,humandev_consensus_full_class_10,humandev_consensus_full_class_11,humandev_consensus_full_class_12,humandev_consensus_full_class_2,humandev_consensus_full_class_3,humandev_consensus_full_class_4,humandev_consensus_full_class_5,humandev_consensus_full_class_6,humandev_consensus_full_class_7,humandev_consensus_full_class_8,humandev_consensus_full_class_9,mineral_fraction,organic_fraction,orgc_stock_mineral,orgc_stock_organic,profile_id,top_cm,longitude,latitude
0,1100.0,1058.0,5128.0,118.0,636.0,-0.000390,0.008097,-2.462457e-10,-0.000033,0.827513,-0.019985,-0.013969,856.5,0.428255,-0.000034,9.0,0.819472,0.000019,3.1250,-0.6250,0.000071,32578.0,937.0,1493.0,1342.0,1515.0,1576.0,1560.0,1579.0,1618.0,32.0,31.0,27.0,28.0,27.0,27.0,29.0,32.0,31.0,34.0,...,14.250000,-7.666667,-7.666667,19.466667,16.850000,-15.6,71.871094,28.0,8.0,28.0,176.0,84.0,203.0,44.599998,1104.151245,0.674167,13886.000000,4.866667,30.0,1964,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,89.0,0.0,0.0,0.0,2.0,0.0,7.00644,0.0,64ND087002_40A1382,0.0,-103.141402,46.351348
1,993.0,707.0,5999.0,65.0,642.0,-0.010228,-0.004593,5.306476e-06,0.000003,-0.572163,-0.760399,-0.430576,193.5,-0.254098,0.000001,7.5,0.555800,0.000008,2.3125,0.1875,0.000022,31930.0,2775.0,1344.0,1355.0,1444.0,1484.0,1569.0,1595.0,1602.0,22.0,20.0,20.0,20.0,22.0,21.0,20.0,22.0,21.0,23.0,...,11.541667,-2.700000,-2.700000,22.933332,20.950001,-9.4,27.701008,150.0,40.0,150.0,296.0,110.0,313.0,39.500000,1034.006104,1.089167,13895.416992,4.041667,30.0,1966,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,15.0,82.0,0.0,0.0,2.0,0.0,14.23080,0.0,S1966IL107001_40A2008,0.0,-89.326942,40.263609
2,1058.0,629.0,5529.0,69.0,737.0,0.012336,0.004595,2.004156e-05,0.000009,0.340314,0.890736,0.526281,204.5,0.235699,0.000014,8.5,0.710805,0.000004,2.5625,0.0000,0.000056,29728.0,2602.0,1325.0,1317.0,1457.0,1521.0,1583.0,1579.0,1581.0,20.0,19.0,18.0,19.0,21.0,21.0,21.0,16.0,17.0,18.0,...,11.916666,0.483333,0.483333,24.083332,12.716667,-6.3,18.270243,196.0,52.0,196.0,266.0,105.0,295.0,37.599998,951.146423,1.181667,14596.333008,3.308333,3

Write to Drive again and download.

In [ ]:
# df.to_csv('/content/drive/MyDrive/soilshot/iscn_env_dataset.csv', index=False)

In [ ]:
# pd.read_csv('/content/drive/MyDrive/soilshot/iscn_env_dataset.csv').head()

In [ ]:
# files.download('/content/drive/MyDrive/soilshot/iscn_env_dataset.csv')

# Accessing the merged dataset

The merged ISCN+covariates dataset is available at https://www.dropbox.com/s/paqtujasjz8lwr8/iscn_env_dataset.csv.

In [38]:
! wget https://www.dropbox.com/s/paqtujasjz8lwr8/iscn_env_dataset.csv

--2021-06-03 03:31:54--  https://www.dropbox.com/s/paqtujasjz8lwr8/iscn_env_dataset.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/paqtujasjz8lwr8/iscn_env_dataset.csv [following]
--2021-06-03 03:31:55--  https://www.dropbox.com/s/raw/paqtujasjz8lwr8/iscn_env_dataset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc11c3ec383a564bc48cdc182f1d.dl.dropboxusercontent.com/cd/0/inline/BPvuF9PCLXjaX75n3TIhhCfBDgHHxjb-doKCGhJJ0uCl6ZBl9JsiG1GcCJn-RwqCs2AV1k-XxG6T-hug7E-E1AK4fE111IoRObxCsYYWPKJST8CMdS9-8Qkoc1igeu2yMEctyy5xzJdHSt3BzerFKp1q/file# [following]
--2021-06-03 03:31:55--  https://uc11c3ec383a564bc48cdc182f1d.dl.dropboxusercontent.com/cd/0/inline/BPvuF9PCLXjaX75n3TIhhCfBDgHHxjb-doKCGhJJ0uCl6ZBl9JsiG1GcCJn-RwqCs2

In [39]:
pd.read_csv('iscn_env_dataset.csv').head()

,EarthEnvCloudCover_MODCF_interannualSD,EarthEnvCloudCover_MODCF_intraannualSD,EarthEnvCloudCover_MODCF_meanannual,EarthEnvCloudCover_MODCF_seasonality_concentration,EarthEnvCloudCover_MODCF_seasonality_theta,EarthEnvTopoMed_1stOrderPartialDerivEW,EarthEnvTopoMed_1stOrderPartialDerivNS,EarthEnvTopoMed_2ndOrderPartialDerivEW,EarthEnvTopoMed_2ndOrderPartialDerivNS,EarthEnvTopoMed_AspectCosine,EarthEnvTopoMed_AspectSine,EarthEnvTopoMed_Eastness,EarthEnvTopoMed_Elevation,EarthEnvTopoMed_Northness,EarthEnvTopoMed_ProfileCurvature,EarthEnvTopoMed_Roughness,EarthEnvTopoMed_Slope,EarthEnvTopoMed_TangentialCurvature,EarthEnvTopoMed_TerrainRuggednessIndex,EarthEnvTopoMed_TopoPositionIndex,EarthEnvTopoMed_VectorRuggednessMeasure,FanEtAl_Depth_to_Water_Table_AnnualMean,SG_Absolute_depth_to_bedrock,SG_Bulk_density_000cm,SG_Bulk_density_005cm,SG_Bulk_density_015cm,SG_Bulk_density_030cm,SG_Bulk_density_060cm,SG_Bulk_density_100cm,SG_Bulk_density_200cm,SG_CEC_000cm,SG_CEC_005cm,SG_CEC_015cm,SG_CEC_030cm,SG_CEC_060cm,SG_CEC_100cm,SG_CEC_200cm,SG_Clay_Content_000cm,SG_Clay_Content_005cm,SG_Clay_Content_015cm,...,WorldClim2_BIO_Mean_Diurnal_Range,WorldClim2_BIO_Mean_Temperature_of_Coldest_Quarter,WorldClim2_BIO_Mean_Temperature_of_Driest_Quarter,WorldClim2_BIO_Mean_Temperature_of_Warmest_Quarter,WorldClim2_BIO_Mean_Temperature_of_Wettest_Quarter,WorldClim2_BIO_Min_Temperature_of_Coldest_Month,WorldClim2_BIO_Precipitation_Seasonality,WorldClim2_BIO_Precipitation_of_Coldest_Quarter,WorldClim2_BIO_Precipitation_of_Driest_Month,WorldClim2_BIO_Precipitation_of_Driest_Quarter,WorldClim2_BIO_Precipitation_of_Warmest_Quarter,WorldClim2_BIO_Precipitation_of_Wettest_Month,WorldClim2_BIO_Precipitation_of_Wettest_Quarter,WorldClim2_BIO_Temperature_Annual_Range,WorldClim2_BIO_Temperature_Seasonality,WorldClim2_H2OVaporPressure_AnnualMean,WorldClim2_SolarRadiation_AnnualMean,WorldClim2_WindSpeed_AnnualMean,bot_cm,datetime,humandev_consensus_full_class_1,humandev_consensus_full_class_10,humandev_consensus_full_class_11,humandev_consensus_full_class_12,humandev_consensus_full_class_2,humandev_consensus_full_class_3,humandev_consensus_full_class_4,humandev_consensus_full_class_5,humandev_consensus_full_class_6,humandev_consensus_full_class_7,humandev_consensus_full_class_8,humandev_consensus_full_class_9,mineral_fraction,organic_fraction,orgc_stock_mineral,orgc_stock_organic,profile_id,top_cm,longitude,latitude
0,1100.0,1058.0,5128.0,118.0,636.0,-0.000390,0.008097,-2.462457e-10,-0.000033,0.827513,-0.019985,-0.013969,856.5,0.428255,-0.000034,9.0,0.819472,0.000019,3.1250,-0.6250,0.000071,32578.0,937.0,1493.0,1342.0,1515.0,1576.0,1560.0,1579.0,1618.0,32.0,31.0,27.0,28.0,27.0,27.0,29.0,32.0,31.0,34.0,...,14.250000,-7.666667,-7.666667,19.466667,16.850000,-15.6,71.871094,28.0,8.0,28.0,176.0,84.0,203.0,44.599998,1104.151245,0.674167,13886.000000,4.866667,30.0,1964,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,89.0,0.0,0.0,0.0,2.0,0.0,7.00644,0.0,64ND087002_40A1382,0.0,-103.141402,46.351348
1,993.0,707.0,5999.0,65.0,642.0,-0.010228,-0.004593,5.306476e-06,0.000003,-0.572163,-0.760399,-0.430576,193.5,-0.254098,0.000001,7.5,0.555800,0.000008,2.3125,0.1875,0.000022,31930.0,2775.0,1344.0,1355.0,1444.0,1484.0,1569.0,1595.0,1602.0,22.0,20.0,20.0,20.0,22.0,21.0,20.0,22.0,21.0,23.0,...,11.541667,-2.700000,-2.700000,22.933332,20.950001,-9.4,27.701008,150.0,40.0,150.0,296.0,110.0,313.0,39.500000,1034.006104,1.089167,13895.416992,4.041667,30.0,1966,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,15.0,82.0,0.0,0.0,2.0,0.0,14.23080,0.0,S1966IL107001_40A2008,0.0,-89.326942,40.263609
2,1058.0,629.0,5529.0,69.0,737.0,0.012336,0.004595,2.004156e-05,0.000009,0.340314,0.890736,0.526281,204.5,0.235699,0.000014,8.5,0.710805,0.000004,2.5625,0.0000,0.000056,29728.0,2602.0,1325.0,1317.0,1457.0,1521.0,1583.0,1579.0,1581.0,20.0,19.0,18.0,19.0,21.0,21.0,21.0,16.0,17.0,18.0,...,11.916666,0.483333,0.483333,24.083332,12.716667,-6.3,18.270243,196.0,52.0,196.0,266.0,105.0,295.0,37.599998,951.146423,1.181667,14596.333008,3.308333,3

# Issues

1. MODIS LAI and NDVI image col

# References

- https://mygeoblog.com/2019/07/17/sample-an-image-collection/